In [7]:
import pandas as pd

# Load the data
student_data_path = './/preprocessed_student_data.csv'
student_skeleton_path = './/student_skeleton.csv'

student_data = pd.read_csv(student_data_path)
student_skeleton = pd.read_csv(student_skeleton_path)


In [9]:
import pandas as pd
import numpy as np

def merge_on_closest_timestamp(skeleton, data, timestamp_column='timestamp', session_column='sessionId'):
    # Ensure data is sorted by session and timestamp
    data = data.sort_values(by=[session_column, timestamp_column])
    skeleton = skeleton.sort_values(by=[session_column, timestamp_column])

    # Create an empty DataFrame to store the merged data
    merged_data = pd.DataFrame()

    for session in skeleton[session_column].unique():
        # Filter rows by session
        skeleton_session = skeleton[skeleton[session_column] == session]
        data_session = data[data[session_column] == session]

        if data_session.empty:
            continue

        # Initialize a column to store the closest timestamps from data_session
        skeleton_session['closest_timestamp'] = np.nan

        # Iterate over the skeleton_session DataFrame and find the closest timestamp in data_session
        for index, row in skeleton_session.iterrows():
            skeleton_timestamp = row[timestamp_column]
            
            # Compute the absolute difference between the skeleton timestamp and all data_session timestamps
            abs_diff = data_session[timestamp_column].sub(skeleton_timestamp).abs()
            
            # Find the index of the minimum difference
            closest_idx = abs_diff.idxmin()

            # Store the closest timestamp in the skeleton_session DataFrame
            skeleton_session.at[index, 'closest_timestamp'] = data_session.loc[closest_idx, timestamp_column]

        # Merge the skeleton_session DataFrame with the data_session DataFrame based on the closest timestamp
        merged_session = pd.merge(skeleton_session, data_session, how='left', left_on=['sessionId', 'closest_timestamp'], right_on=[session_column, timestamp_column])

        # Append the merged_session DataFrame to the merged_data DataFrame
        merged_data = pd.concat([merged_data, merged_session])

    # Drop the 'closest_timestamp' column if no longer needed
    merged_data.drop(columns=['closest_timestamp', 'affect_x', 'context_x','notification', 'engagement', 'affect_y', 'context_y'], inplace=True)
    
    return merged_data

merged_data = merge_on_closest_timestamp(student_skeleton, student_data)



C:\Users\tkoehler\AppData\Local\Temp\1\ipykernel_33476\619494597.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skeleton_session['closest_timestamp'] = np.nan
C:\Users\tkoehler\AppData\Local\Temp\1\ipykernel_33476\619494597.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skeleton_session['closest_timestamp'] = np.nan
C:\Users\tkoehler\AppData\Local\Temp\1\ipykernel_33476\619494597.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [12]:
features_for_prediction = ['x', 'y', 'z', 'ppgValue', 'hr', 'hrIbi', 'hrStatus', 'ibiStatus']
X_for_prediction = merged_data[features_for_prediction]

from joblib import dump, load

# Load the model from the file
clf_loaded = load('random_forest_model.joblib')

# Now you can use clf_loaded to make predictions or evaluate it
predictions = clf_loaded.predict(X_for_prediction)

# Attach predictions to the merged_data DataFrame
merged_data['predicted_context'] = predictions[:,0]
merged_data['predicted_affect'] = predictions[:,1]

In [14]:
student_skeleton_prediction = student_skeleton
student_skeleton_prediction['context'] = predictions[:,0]
student_skeleton_prediction['affect'] = predictions[:,1]


#Change name of file here!
student_skeleton_prediction.to_csv('prediction_random_forest.csv', index=False)
